# 🧠 Understanding Function Calling with OpenAI API (Function Tool Calling)

In this lesson, we explore **Function Calling** with OpenAI's API — a powerful feature that allows the model to call specific functions based on user intent. This makes the model useful for building **AI agents**, **chatbots**, and **smart automation systems**.

---

## 🔧 What is Function Calling?

Function calling lets the AI decide **when** and **how** to call an external function you've defined — based on the user's query. You define the function's name, parameters, and descriptions, and the model takes care of the rest.

---

## 🌦️ Example 1: `get_weather` Function

This example shows how to define a simple function that gets the weather for a given location.

### 🔍 Code:

```python
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": ["location"],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(response.output)



### 🧠 Explanation:

* You define a tool named `get_weather` with one required parameter: `location`.
* The user asks about the weather in Paris.
* The model recognizes this intent and responds by calling the function with `{"location": "Paris"}`.
* The result would be passed back to the user (from a real API or mock function).



## 📧 Example 2: `send_email` Function

Now, let’s see how we define a tool for sending an email.

### 🔍 Code:

```python
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "send_email",
    "description": "Send an email to a recipient with a subject and body.",
    "parameters": {
        "type": "object",
        "properties": {
            "recipient": {
                "type": "string",
                "description": "The recipient's email address."
            },
            "subject": {
                "type": "string",
                "description": "The subject of the email."
            },
            "body": {
                "type": "string",
                "description": "The body content of the email."
            }
        },
        "required": ["recipient", "subject", "body"],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "Send an email to john@example.com about the meeting."}],
    tools=tools
)

print(response.output)
```

### 🧠 Explanation:

* The `send_email` tool requires 3 fields: `recipient`, `subject`, and `body`.
* The model will extract this information from the user's request and call the function accordingly.
* This could be used to automate real-world messaging systems with natural language.


In [1]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai
!pip install python-dotenv

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [3]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    


## Example: Weather Function

This is a corrected and complete version of the provided code snippet. We'll define a mock weather function.

In [4]:
import json

# Define the mock function
def get_weather(location):
    # Mock response; in real scenario, call an API
    return f"The current temperature in {location} is 72°F."

# Define the tool
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Bogotá, Colombia"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# User query
messages = [{"role": "user", "content": "What is the weather like in Paris today?"}]

# First API call
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)



# Print model's response
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Kpy9D2E4kHNUPw1LZqMzgOST', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')])


In [5]:
response_message.tool_calls

[ChatCompletionMessageToolCall(id='call_Kpy9D2E4kHNUPw1LZqMzgOST', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]

In [6]:
response_message.tool_calls[0].function

Function(arguments='{"location":"Paris, France"}', name='get_weather')

# Apply to the `get_weather` function to simulate a weather API call.


In [7]:
tool_calls = response_message.tool_calls

# Handle tool calls
if tool_calls:
    messages.append(response_message)  # Append assistant's message
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name == "get_weather":
            location = function_args["location"]
            function_response = get_weather(location)
        
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": function_response
        })
    
    # Second API call for final response
    second_response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    print(second_response.choices[0].message.content)

The current temperature in Paris, France is 72°F.


# Evalution with questions

In [8]:
import openai
import json

# Define the mock function
def get_weather(location):
    # Mock response; in real scenario, call an API
    return f"The current temperature in {location} is 72°F."

# Define the tool
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Bogotá, Colombia"
                    }
                },
                "required": ["location"]
            }
        }
    }
]



client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    


In [9]:
# List of questions
questions = [
    # คำถามที่เรียกใช้ฟังก์ชัน get_weather
    "ตอนนี้ที่โตเกียว ประเทศญี่ปุ่น อุณหภูมิเท่าไหร่?",
    "ช่วยบอกสภาพอากาศที่ไนโรบี ประเทศเคนยาให้หน่อย",
    "วันนี้กรุงเทพฯ อากาศร้อนแค่ไหน?",
    "ผมจะไปเที่ยวเบอร์ลิน ประเทศเยอรมนี อากาศเป็นยังไงบ้าง?",
    "ตอนนี้ที่ซิดนีย์ ประเทศออสเตรเลีย อากาศเป็นอย่างไร?",
    
    # คำถามทั่วไป
    "ใครเป็นประธานาธิบดีของฝรั่งเศสคนปัจจุบัน?",
    "อธิบายกระบวนการสังเคราะห์แสงให้หน่อย",
    "เมืองหลวงของประเทศแคนาดาคือเมืองอะไร?",
    "ช่วยสรุปเนื้อเรื่องของโรมิโอและจูเลียตหน่อย",
    "ข้าวผัดทำอย่างไร?"
]

questions

['ตอนนี้ที่โตเกียว ประเทศญี่ปุ่น อุณหภูมิเท่าไหร่?',
 'ช่วยบอกสภาพอากาศที่ไนโรบี ประเทศเคนยาให้หน่อย',
 'วันนี้กรุงเทพฯ อากาศร้อนแค่ไหน?',
 'ผมจะไปเที่ยวเบอร์ลิน ประเทศเยอรมนี อากาศเป็นยังไงบ้าง?',
 'ตอนนี้ที่ซิดนีย์ ประเทศออสเตรเลีย อากาศเป็นอย่างไร?',
 'ใครเป็นประธานาธิบดีของฝรั่งเศสคนปัจจุบัน?',
 'อธิบายกระบวนการสังเคราะห์แสงให้หน่อย',
 'เมืองหลวงของประเทศแคนาดาคือเมืองอะไร?',
 'ช่วยสรุปเนื้อเรื่องของโรมิโอและจูเลียตหน่อย',
 'ข้าวผัดทำอย่างไร?']

In [10]:



# Loop through each question to test the code
for question in questions:
    print(f"\nTesting question: {question}")
    
    # User query
    messages = [{"role": "user", "content": question}]
    
    # First API call to check if tool is needed
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # Get model's response
    response_message = response.choices[0].message
    print("Initial response message:")
    print(response_message)
    
    # Check if tool call is triggered
    if response_message.tool_calls:
        print("Tool call detected. Executing the tool...")
        
        # Assuming single tool call for simplicity
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name == "get_weather":
            # Call the mock function
            function_response = get_weather(function_args["location"])
            
            # Append the tool response to messages
            messages.append(response_message)  # Append the assistant's message with tool call
            messages.append({
                "role": "tool",
                "content": function_response,
                "tool_call_id": tool_call.id
            })
            
            # Second API call to get the final response
            second_response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )
            
            final_message = second_response.choices[0].message
            print("Final response after tool execution:")
            print(final_message.content)
        else:
            print("Unknown tool called.")
    else:
        # No tool call, direct response
        print("No tool call. Direct response:")
        print(response_message.content)



Testing question: ตอนนี้ที่โตเกียว ประเทศญี่ปุ่น อุณหภูมิเท่าไหร่?
Initial response message:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_s0JPQjUdz4VOw8zJBv8HSqfu', function=Function(arguments='{"location":"โตเกียว, ประเทศญี่ปุ่น"}', name='get_weather'), type='function')])
Tool call detected. Executing the tool...
Final response after tool execution:
ตอนนี้ที่โตเกียว ประเทศญี่ปุ่น อุณหภูมิอยู่ที่ 72°F ครับ/ค่ะ

Testing question: ช่วยบอกสภาพอากาศที่ไนโรบี ประเทศเคนยาให้หน่อย
Initial response message:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GEqgJP0jdubvHqrEOpNcydGM', function=Function(arguments='{"location":"Nairobi, Kenya"}', name='get_weather'), type='function')])
Tool call detected. Executing the tool...
Final response after tool execution

## Practice Questions

For each of the following 10 questions, define the appropriate tool(s), mock function(s), and implement the full function calling flow. Test with the given user query.

### Question 1: Currency Conversion
Create a tool to convert an amount from one currency to another (e.g., USD to EUR). Mock the conversion rate. Query: "Convert 100 USD to EUR."

In [11]:
# Your code here

### Question 2: Current Time in Timezone
Create a tool to get the current time in a specific timezone. Use Python's datetime for mock. Query: "What time is it in Tokyo?"

In [12]:
# Your code here

### Question 3: Stock Price Lookup
Create a tool to get the current stock price for a company. Mock the price. Query: "What's the stock price of Apple?"

In [13]:
# Your code here

### Question 4: Math Calculation
Create a tool to evaluate a math expression. Use eval() for mock (be careful). Query: "What is 15 * 12?"

In [14]:
# Your code here

### Question 5: Get a Random Joke
Create a tool to fetch a random joke. Mock with a list of jokes. Query: "Tell me a joke."

In [15]:
# Your code here

### Question 6: Text Translation
Create a tool to translate text from English to Spanish. Mock with simple dictionary or fixed response. Query: "Translate 'Hello, how are you?' to Spanish."

In [16]:
# Your code here

### Question 7: Capital of a Country
Create a tool to get the capital city of a country. Mock with a dictionary. Query: "What is the capital of Japan?"

In [17]:
# Your code here

### Question 8: Add Two Numbers
Create a tool to add two numbers. Query: "Add 45 and 67."

In [18]:
# Your code here

### Question 9: Latest News Headline
Create a tool to get the latest news on a topic. Mock response. Query: "What's the latest news on AI?"

In [19]:
# Your code here

### Question 10: Get User Email
Create a tool to retrieve a user's email by username. Mock with dictionary. Query: "What's the email for user john_doe?"

In [20]:
# Your code here